In [1]:
#import packages

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report, recall_score, precision_score
from sklearn.pipeline import make_pipeline

from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# filter warning messages
import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
# import data set and create a data frame
df_dataset = pd.read_csv('http://dl.dropboxusercontent.com/s/xn2a4kzf0zer0xu/acquisition_train.csv?dl=0')

# dataset shape
df_dataset.shape
df_dataset.head(2)

(45000, 43)

,ids,target_default,score_1,score_2,score_3,score_4,score_5,score_6,risk_rate,last_amount_borrowed,last_borrowed_in_months,credit_limit,reason,income,facebook_profile,state,zip,channel,job_name,real_state,ok_since,n_bankruptcies,n_defaulted_loans,n_accounts,n_issues,application_time_applied,application_time_in_funnel,email,external_data_provider_credit_checks_last_2_year,external_data_provider_credit_checks_last_month,external_data_provider_credit_checks_last_year,external_data_provider_email_seen_before,external_data_provider_first_name,external_data_provider_fraud_score,lat_lon,marketing_channel,profile_phone_number,reported_income,shipping_state,shipping_zip_code,profile_tags,user_agent,target_fraud
0,343b7e7b-2cf8-e508-b8fd-0a0285af30aa,False,1Rk8w4Ucd5yR3KcqZzLdow==,IOVu8au3ISbo6+zmfnYwMg==,350.0,101.800832,0.259555,108.427273,0.40,25033.92,36.0,0.0,mLVIVxoGY7TUDJ1FyFoSIZi1SFcaBmO01AydRchaEiGYtU...,65014.12,True,sjJbkqJS7cXalHLBFA+EOQ==,Ernn+uVXCMq/6ARrBCcd+A==,NCqL3QBx0pscDnx3ixKwXg==,mLVIVxoGY7TUDJ1FyFoSIZi1SFcaBmO01AydRchaEiGYtU...,N5/CE7lSkAfB04hVFFwllw==,NaN,0.0,0.0,18.0,18.0,07:52:34,444,outlook.com,NaN,2,0.0,51.0,leidelaura,645,"(-29.151545708122246, -51.1386461804385)",Invite-email,514-9840782,57849.0,BR-MT,17528,"{'tags': ['n19', 'n8']}",Mozilla/5.0 (Linux; Android 6.0.1; SGP771 Buil...,NaN
1,bc2c7502-bbad-0f8c-39c3-94e881967124,False,DGCQep2AE5QRkNCshIAlFQ==,SaamrHMo23l/3TwXOWgVzw==,370.0,97.062615,0.942655,92.002546,0.24,NaN,NaN,39726.0,mLVIVxoGY7TUDJ1FyFoSIZi1SFcaBmO01AydRchaEiGYtU...,100018.91,False,xsd3ZdsI3356I3xMxZeiqQ==,rlWIXTBO+VOa34+SpGyhlQ==,NCqL3QBx0pscDnx3ixKwXg==,mLVIVxoGY7TUDJ1FyFoSIZi1SFcaBmO01AydRchaEiGYtU...,N5/CE7lSkAfB04hVFFwllw==,50.0,0.0,0.0,14.0,14.0,02:34:29,346,gmail.com,0.0,1,0.0,17.0,diocezio,243,"(-19.687710705798963, -47.94151536525154)",Radio-commercial,251-3659293,4902.0,BR-RS,40933,"{'tags': ['n6', 'n7', 'nim']}",Mozilla/5.0 (Linux; Android 5.0.2; SAMSUNG SM-...,NaN


In [3]:
# dataset summary
df_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 43 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   ids                                               45000 non-null  object 
 1   target_default                                    41741 non-null  object 
 2   score_1                                           44438 non-null  object 
 3   score_2                                           44438 non-null  object 
 4   score_3                                           44438 non-null  float64
 5   score_4                                           45000 non-null  float64
 6   score_5                                           45000 non-null  float64
 7   score_6                                           45000 non-null  float64
 8   risk_rate                                         44438 non-null  float64
 9   last_amount_borro

In [4]:
# percentage of missing values per feature
(df_dataset.isnull().sum() * 100 / df_dataset.shape[0]).round(2).sort_values(ascending=False)

target_fraud                                        96.62
last_borrowed_in_months                             66.57
last_amount_borrowed                                66.57
ok_since                                            58.99
external_data_provider_credit_checks_last_2_year    50.28
external_data_provider_credit_checks_last_year      33.61
credit_limit                                        30.67
n_issues                                            25.65
facebook_profile                                     9.91
marketing_channel                                    7.95
job_name                                             7.41
target_default                                       7.24
external_data_provider_email_seen_before             4.96
lat_lon                                              3.03
user_agent                                           1.60
n_bankruptcies                                       1.55
n_defaulted_loans                                    1.28
reason        

In [5]:
## target_default is the DV -> we need only the data which have DV for training and testing
df_dataset.dropna(subset=['target_default'], inplace=True)
df_dataset.shape

(41741, 43)

In [6]:
# drop the column "target_fraud" as the missing % is very high
df_dataset.drop('target_fraud', axis=1, inplace=True)
df_dataset.shape

(41741, 42)

In [7]:
#no. of unique values of the features
df_dataset.nunique().sort_values()

external_data_provider_credit_checks_last_2_year        1
channel                                                 1
target_default                                          2
last_borrowed_in_months                                 2
facebook_profile                                        2
external_data_provider_credit_checks_last_year          2
external_data_provider_credit_checks_last_month         4
n_defaulted_loans                                       5
real_state                                              5
email                                                   6
n_bankruptcies                                          6
score_1                                                 7
marketing_channel                                       9
shipping_state                                         25
score_2                                                35
n_issues                                               44
n_accounts                                             44
state         

In [8]:
df_dataset['channel'].isnull().sum()/df_dataset.shape[0]*100

np.float64(0.0)

In [9]:
# drop the columns "channel" (0% missing and all the values are the same)
df_dataset.drop(labels=['channel'], axis=1, inplace=True)

In [10]:
df_dataset['external_data_provider_credit_checks_last_2_year'].isnull().sum()/df_dataset.shape[0]*100

np.float64(50.382118300951106)

In [11]:
# drop the columns "external_data_provider_credit_checks_last_2_year" (50% missing and remaining all are zero(no checks) -> meaning no data or no checks)
df_dataset.drop(labels=['external_data_provider_credit_checks_last_2_year'], axis=1, inplace=True)

In [12]:
# Columns that are not useful for training the data

df_dataset.drop(labels=['email', 'reason', 'zip', 'job_name', 'external_data_provider_first_name', 'lat_lon',
                       'shipping_zip_code', 'user_agent', 'profile_tags', 'marketing_channel',
                       'profile_phone_number', 'application_time_applied', 'ids'], axis=1, inplace=True)

In [13]:
# show descriptive statistics
df_dataset.describe()

,score_3,score_4,score_5,score_6,risk_rate,last_amount_borrowed,last_borrowed_in_months,credit_limit,income,ok_since,n_bankruptcies,n_defaulted_loans,n_accounts,n_issues,application_time_in_funnel,external_data_provider_credit_checks_last_month,external_data_provider_credit_checks_last_year,external_data_provider_email_seen_before,external_data_provider_fraud_score,reported_income
count,41741.000000,41741.000000,41741.000000,41741.000000,41741.000000,14133.000000,14133.000000,28632.000000,4.174100e+04,17276.000000,41606.000000,41729.000000,41741.000000,30818.000000,41741.000000,41741.000000,27720.000000,39656.000000,41741.000000,41741.0
mean,346.459836,100.006820,0.499416,99.919399,0.294451,13328.104095,40.588410,33877.220453,7.108012e+04,35.192174,0.076696,0.004625,10.639108,11.023882,247.748545,1.504396,0.504185,12.731188,500.491771,inf
std,110.102271,3.183821,0.288085,10.022703,0.101561,7918.698433,9.437936,36141.985884,5.225978e+04,21.629577,0.274820,0.080157,4.588175,4.596036,146.326172,1.114207,0.499992,125.711218,287.993121,NaN
min,0.000000,86.191572,0.000035,60.663039,0.000000,1005.180000,36.000000,0.000000,4.821180e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-999.000000,0.000000,403.0
25%,270.000000,97.862546,0.251595,93.182517,0.220000,7210.280000,36.000000,9975.000000,4.401958e+04,17.000000,0.000000,0.000000,7.000000,8.000000,120.000000,1.000000,0.000000,11.000000,252.000000,50910.0
50%,340.000000,100.017950,0.500174,99.977774,0.290000,12011.050000,36.000000,25213.000000,6.004409e+04,32.000000,0.000000,0.000000,10.000000,10.000000,248.000000,2.000000,1.000000,27.000000,502.000000,101623.0
75%,420.000000,102.143100,0.747630,106.630991,0.360000,18030.160000,36.000000,46492.500000,8.503289e+04,50.000000,0.000000,0.000000,13.000000,14.000000,375.000000,2.000000,1.000000,43.000000,747.000000,151248.0
max,990.000000,113.978234,0.999973,142.192400,0.900000,35059.600000,60.000000,448269.000000,5.000028e+06,141.000000,5.000000,5.000000,49.000000,49.000000,500.000000,3.000000,1.000000,59.000000,1000.000000,inf


In [14]:
# count of "inf" values in "reported_income"
np.isinf(df_dataset['reported_income']).sum()

np.int64(66)

In [15]:
# count of values = -999 (special values) in "external_data_provider_email_seen_before"
df_dataset.loc[df_dataset['external_data_provider_email_seen_before'] == -999, 'external_data_provider_email_seen_before'].value_counts()

external_data_provider_email_seen_before
-999.0    591
Name: count, dtype: int64

In [16]:
# Lets replace "inf" and "-999" values with "nan"
df_dataset['reported_income'] = df_dataset['reported_income'].replace(np.inf, np.nan)
df_dataset.loc[df_dataset['external_data_provider_email_seen_before'] == -999, 'external_data_provider_email_seen_before'] = np.nan

In [17]:
# dataset summary
df_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41741 entries, 0 to 44999
Data columns (total 27 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   target_default                                   41741 non-null  object 
 1   score_1                                          41741 non-null  object 
 2   score_2                                          41741 non-null  object 
 3   score_3                                          41741 non-null  float64
 4   score_4                                          41741 non-null  float64
 5   score_5                                          41741 non-null  float64
 6   score_6                                          41741 non-null  float64
 7   risk_rate                                        41741 non-null  float64
 8   last_amount_borrowed                             14133 non-null  float64
 9   last_borrowed_in_months          

In [18]:
df_dataset.head(2)

,target_default,score_1,score_2,score_3,score_4,score_5,score_6,risk_rate,last_amount_borrowed,last_borrowed_in_months,credit_limit,income,facebook_profile,state,real_state,ok_since,n_bankruptcies,n_defaulted_loans,n_accounts,n_issues,application_time_in_funnel,external_data_provider_credit_checks_last_month,external_data_provider_credit_checks_last_year,external_data_provider_email_seen_before,external_data_provider_fraud_score,reported_income,shipping_state
0,False,1Rk8w4Ucd5yR3KcqZzLdow==,IOVu8au3ISbo6+zmfnYwMg==,350.0,101.800832,0.259555,108.427273,0.40,25033.92,36.0,0.0,65014.12,True,sjJbkqJS7cXalHLBFA+EOQ==,N5/CE7lSkAfB04hVFFwllw==,NaN,0.0,0.0,18.0,18.0,444,2,0.0,51.0,645,57849.0,BR-MT
1,False,DGCQep2AE5QRkNCshIAlFQ==,SaamrHMo23l/3TwXOWgVzw==,370.0,97.062615,0.942655,92.002546,0.24,NaN,NaN,39726.0,100018.91,False,xsd3ZdsI3356I3xMxZeiqQ==,N5/CE7lSkAfB04hVFFwllw==,50.0,0.0,0.0,14.0,14.0,346,1,0.0,17.0,243,4902.0,BR-RS


In [19]:
# dataset containing numerical features
df_dataset_numerical = df_dataset.select_dtypes(
    include=['int64', 'float64']
)
df_dataset_numerical.shape

(41741, 20)

In [20]:
df_dataset_num = df_dataset.select_dtypes(exclude='object').columns
df_dataset_cat = df_dataset.select_dtypes(include='object').columns

# fill missing values for "last_amount_borrowed", "last_borrowed_in_months" and "n_issues" as 0  bcz not every client would have values assigned to these variables.
df_dataset['last_amount_borrowed'].fillna(value=0, inplace=True)
df_dataset['last_borrowed_in_months'].fillna(value=0, inplace=True)
df_dataset['n_issues'].fillna(value=0, inplace=True)

In [21]:
# fill missing values for numerical variables with median
imputer = SimpleImputer(missing_values=np.nan, strategy='median')
imputer = imputer.fit(df_dataset.loc[:, df_dataset_num])
df_dataset.loc[:, df_dataset_num] = imputer.transform(df_dataset.loc[:, df_dataset_num])

In [22]:
# fill missing values for categorical variables with most recuuring
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputer = imputer.fit(df_dataset.loc[:, df_dataset_cat])
df_dataset.loc[:, df_dataset_cat] = imputer.transform(df_dataset.loc[:, df_dataset_cat])

In [23]:
df_dataset.isnull().sum()

target_default                                     0
score_1                                            0
score_2                                            0
score_3                                            0
score_4                                            0
score_5                                            0
score_6                                            0
risk_rate                                          0
last_amount_borrowed                               0
last_borrowed_in_months                            0
credit_limit                                       0
income                                             0
facebook_profile                                   0
state                                              0
real_state                                         0
ok_since                                           0
n_bankruptcies                                     0
n_defaulted_loans                                  0
n_accounts                                    

In [24]:
## Now converting cat values to numericals
df_dataset.shape

bin_var = df_dataset.nunique()[df_dataset.nunique() == 2].keys().tolist()
num_var = [col for col in  df_dataset.select_dtypes(['int', 'float']).columns.tolist() if col not in bin_var]
cat_var = [col for col in  df_dataset.select_dtypes(['object']).columns.tolist() if col not in bin_var]

bin_var
num_var
cat_var

(41741, 27)

['target_default',
 'facebook_profile',
 'external_data_provider_credit_checks_last_year']

['score_3',
 'score_4',
 'score_5',
 'score_6',
 'risk_rate',
 'last_amount_borrowed',
 'last_borrowed_in_months',
 'credit_limit',
 'income',
 'ok_since',
 'n_bankruptcies',
 'n_defaulted_loans',
 'n_accounts',
 'n_issues',
 'application_time_in_funnel',
 'external_data_provider_credit_checks_last_month',
 'external_data_provider_email_seen_before',
 'external_data_provider_fraud_score',
 'reported_income']

['score_1', 'score_2', 'state', 'real_state', 'shipping_state']

In [25]:
df_dataset_encoded = df_dataset.copy()

# label encoding for the binary variables
label_encoder = LabelEncoder()
for col in bin_var:
  df_dataset_encoded[col] = label_encoder.fit_transform(df_dataset_encoded[col])

# encoding with get_dummies for the categorical variables
df_dataset_encoded = pd.get_dummies(df_dataset_encoded, columns=cat_var)
df_dataset_encoded.shape
df_dataset_encoded.head()

(41741, 144)

,target_default,score_3,score_4,score_5,score_6,risk_rate,last_amount_borrowed,last_borrowed_in_months,credit_limit,income,facebook_profile,ok_since,n_bankruptcies,n_defaulted_loans,n_accounts,n_issues,application_time_in_funnel,external_data_provider_credit_checks_last_month,external_data_provider_credit_checks_last_year,external_data_provider_email_seen_before,external_data_provider_fraud_score,reported_income,score_1_1Rk8w4Ucd5yR3KcqZzLdow==,score_1_4DLlLW62jReXaqbPaHp1vQ==,score_1_8k8UDR4Yx0qasAjkGrUZLw==,score_1_DGCQep2AE5QRkNCshIAlFQ==,score_1_e4NYDor1NOw6XKGE60AWFw==,score_1_fyrlulOiZ+5hoFqLa6UbDQ==,score_1_smzX0nxh5QlePvtVf6EAeg==,score_2_+2hzpeP1RWr8PEvL1WTUdw==,score_2_+CxEO4w7jv3QPI/BQbyqAA==,score_2_/tdlnWjXoZ3OjdtBXzdOJQ==,score_2_5/uMrqKj3OL/Xk5OrGx9fg==,score_2_55UK234RR1d7HIWJjmq9tw==,score_2_6J1ZMTzN5GKHXnhM4J1JbA==,score_2_7h+tk4z7O9brtBSe1rNjxA==,score_2_7h8PTkrlTWUPP3yuyP4rUg==,score_2_A+QuW1n/ABeiVVe/9CRZ9Q==,score_2_Fv28Bz0YRTVAT5kl1bAV6g==,score_2_IOVu8au3ISbo6+zmfnYwMg==,score_2_LCak332j+TYFqHC3NDwiqg==,score_2_NLvAOzzmJba/0zolQnWF5Q==,score_2_O4i7FxcROACMVTCgI0WXuA==,score_2_OlDYtdljgSSYM/M1L2CRaQ==,score_2_RO7MTL+j4PH2gNzbhNTq/A==,score_2_SaamrHMo23l/3TwXOWgVzw==,score_2_YLGMUI9hObSh6wD/xfanGg==,score_2_bopP0NxW3+r8tn9xIHTaOw==,score_2_cdpgyOyZS04uXerMNu7uCw==,score_2_d/7Hedyz7ovK9Pn1CYN4+A==,score_2_dCm9hFKfdRm7ej3jW+gyxw==,score_2_dWJRASUFMejk3AHZ1p1Gkg==,score_2_emS9xH8CLoRNie2uSmaDAQ==,score_2_ky19q4V1ZqgL3jnHX0wKDw==,score_2_mX2VRRG38RPiHX+MfjefRw==,score_2_osCzpM4hJrxugqWWuZmMWw==,score_2_pAzpxkhjPsjWldgSX21+zg==,score_2_rJZgTmANW3PjOCQLCcp4iQ==,score_2_tHpS8e9F8d9zg3iOQM9tsA==,score_2_tQUTfUyeuGkhRotd+6WjVg==,score_2_vJyc9xom9v7hwFMPTIpmKw==,score_2_w1miZqhB5+RSamEQJa0rqg==,score_2_wjdj2vxjWoDsEIk0l09ynw==,score_2_wkeCdGeu5sEv4/fjwR0aDg==,state_/+QaZYcpPt5mXLpkv6I8Bw==,state_/78Q+NAqnsdbiE2JReYK9Q==,state_/EoxQEzy1PWWTtv6DNwr2w==,state_/L8vvVesB5WyAv190Hw/rQ==,state_0gQEZgRHqS5KSF3oWQrQfw==,state_0lQuvmygO68ZFXZjzHEOww==,state_1DpYl6dtzY0qE33poow3iw==,state_1tPrGp5Yx4Yev8EWxfJw2w==,state_1vMmtGURgMcjxQbV2KQiUw==,state_1xxlFOWSFxK0/J2VdCKP3g==,state_5hPtRY8gW/3+fPXvGIgWTg==,state_7bX4XFXmYqoMspxup0CrKA==,state_82aTqSzrTImY+SbAOBQzGQ==,state_8ZKN5LHxobLWh6drCDlmhQ==,state_9PWmQ/Uv20r8XvcwDXWUSw==,state_9sS116y/1CqWLVJ8nFOGZg==,state_BB/zpwTH+8GCIVHlhzOU1Q==,state_DLK5rpOxZhCM5oQvfUpkTQ==,state_E8shyM/6btBNXXExsO17zw==,state_GFOaXqGcKJxSSF9aBDDNaA==,state_GW2VZ3dN3OGHSjQ6JkfqQw==,state_Jc0dzuliQSKFl0Xy0bjYCQ==,state_JrdZzPZEa/YIIGwh8OdTKA==,state_KET/Pmr6rHp1RJ/P9ymztw==,state_LXZkF0tV1Ma9JIlIvaUDiw==,state_NX0S6z5x5fEStwXfBQN5FQ==,state_OCoEDgPLpDjHcbEmPpAjvA==,state_P1TKs1aii8osbgO+MszZ2w==,state_QCVqG0wT4xPLQuiTIExO2w==,state_QMl/wADluYmoYZIXDOUktQ==,state_TYC58USu+Tq7c/WS4YLnuQ==,state_VafbDA6D5W2TpGMAUJbW8Q==,state_Ygq6MsM98oC8yceExr69Ig==,state_ZUmHOp1/xBWh5O/uqDutsw==,state_ZVfsutsd1Go0zZEnGp0vEA==,state_bNDXnbe7XAx/VUe+cbjDiw==,state_bwNrC22VlFOFk6/nEHQc6A==,state_cd+7k16vaf6SzwSyvT/fHw==,state_fMOpZ7DFhwQgTxW547UMCQ==,state_fR2lwAMjCb/fNXjsApiOTw==,state_kfFcVGcsJmH2n3eYprLkeQ==,state_orU7WJYGPT/xFwOXY1eBkw==,state_qItmSqWrjm+MopDA+UpKRw==,state_rrVbgRNTRyB6auEycsRrpA==,state_sjJbkqJS7cXalHLBFA+EOQ==,state_umkEte5GXWggFkMOtKQ3Ug==,state_x6Gw/1I8t3KPgT6s82Kodg==,state_xsd3ZdsI3356I3xMxZeiqQ==,state_ygE7OTdsxRMLeXMa0EZwZw==,state_yni6lL4oSly97UVZeRSyxA==,real_state_+qWF9pJpVGtTFn4vFjb/cg==,real_state_N5/CE7lSkAfB04hVFFwllw==,real_state_UX7AdFYgQh+VrVC5eIaU9w==,real_state_n+xK9CfX0bCn77lClTWviw==,real_state_nSpvDsIsslUaX6GE6m6eQA==,shipping_state_BR-AC,shipping_state_BR-AL,shipping_state_BR-AM,shipping_state_BR-AP,shipping_state_BR-BA,shipping_state_BR-CE,shipping_state_BR-DF,shipping_state_BR-ES,shipping_state_BR-GO,shipping_state_BR-MA,shipping_state_BR-MG,shipping_state_BR-MS,shipping_state_BR-MT,shipping_state_BR-PA,shipping_state_BR-PB,shipping_state_BR-PE,shipping_state_BR-PR,shipping_state_BR-RN,shipping_state_BR-RO,shipping_state_BR-RR,shipping_state_BR-RS,shipping_state_BR-SC,shippi

In [26]:
## Training the model

# feature matrix
X = df_dataset_encoded.drop('target_default', axis=1)

# target vector
y = df_dataset_encoded['target_default']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, shuffle=True, stratify=y, test_size=0.25, random_state=25
)

X_dev, X_val, y_dev, y_val = train_test_split(
    X_train, y_train, shuffle=True, stratify=y_train, test_size=0.25, random_state=25
)

In [27]:
def max_ks(DV, predict_prob):
    merged = np.dstack((predict_prob, DV))
    merged = merged[0]
    mergesort = merged[np.argsort(merged[:,0])]
    total_bad = np.unique(mergesort[:,1], return_counts=True)[1][0]
    total_good = np.unique(mergesort[:,1], return_counts=True)[1][1]
    count0 = 0
    count1 = 0
    max_ks = 0
    for i,j in mergesort:
        if j==1:
            count1 += 1
        elif j==0:
            count0 += 1
        ks = abs(count1 / float(total_good) - count0/float(total_bad))*100

        if ks > max_ks:
            max_ks = ks
    return max_ks

def ks_scorer(estimator, X, y):
    y_pred_proba = estimator.predict_proba(X)[:, 1]
    return max_ks(y.values if hasattr(y, "values") else y,
                        y_pred_proba)

In [28]:
# Define models
models = {
    # 'Logistic Regression': LogisticRegression(max_iter=100, random_state=25),
    # 'Decision Tree': DecisionTreeClassifier(random_state=25),
    # 'Random Forest': RandomForestClassifier(random_state=25),
    # 'Support Vector Classifier': SVC(probability=True, random_state=25),
    # 'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),
    # 'Naive Bayes': GaussianNB(),
    # 'Stochastic Gradient Descent': SGDClassifier(random_state=25), 
    # "Gradient Boosting": GradientBoostingClassifier(random_state=25), 
    # "AdaBoost": AdaBoostClassifier(random_state=25),
    'XGBoost': XGBClassifier(random_state=25), 
    "LightGBM": LGBMClassifier(random_state=25),
    "CatBoost": CatBoostClassifier(random_state=25)
}

results = []
trained_model = {}
for name, model in models.items():
    # Train the model
    model.fit(X_dev, y_dev)
    
    # Predict on validation set
    # Get probability scores
    if hasattr(model, "predict_proba"):
        y_val_score = model.predict_proba(X_val)[:, 1]
    else:
        # For models like SGDClassifier
        y_val_score = model.decision_function(X_val)

    y_val_pred = model.predict(X_val)
    
    trained_model[name] = model 
    
    print("Model: ", name)
    # print("Accuracy: ", accuracy_score(y_val, y_val_pred))
    # print("F1: ", f1_score(y_val, y_val_pred))
    # print("Precision: ", precision_score(y_val, y_val_pred))
    # print("Recall: ", recall_score(y_val, y_val_pred))
    print("Max KS: ", max_ks(y_val, y_val_score))
    print("\\n")

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

Model:  XGBoost
Max KS:  27.750470367172426
\n
[LightGBM] [Info] Number of positive: 3747, number of negative: 19731
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003726 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2917
[LightGBM] [Info] Number of data points in the train set: 23478, number of used features: 135
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.159596 -> initscore=-1.661235
[LightGBM] [Info] Start training from score -1.661235


LGBMClassifier(random_state=25)

Model:  LightGBM
Max KS:  30.81926775838427
\n
Learning rate set to 0.039647
0:	learn: 0.6686021	total: 135ms	remaining: 2m 15s
1:	learn: 0.6454354	total: 146ms	remaining: 1m 12s
2:	learn: 0.6257112	total: 155ms	remaining: 51.6s
3:	learn: 0.6079339	total: 165ms	remaining: 41.1s
4:	learn: 0.5912622	total: 175ms	remaining: 34.8s
5:	learn: 0.5754265	total: 184ms	remaining: 30.5s
6:	learn: 0.5615320	total: 195ms	remaining: 27.6s
7:	learn: 0.5485932	total: 204ms	remaining: 25.2s
8:	learn: 0.5361497	total: 214ms	remaining: 23.6s
9:	learn: 0.5248979	total: 224ms	remaining: 22.2s
10:	learn: 0.5144061	total: 234ms	remaining: 21s
11:	learn: 0.5066214	total: 244ms	remaining: 20.1s
12:	learn: 0.4974914	total: 254ms	remaining: 19.3s
13:	learn: 0.4909960	total: 263ms	remaining: 18.5s
14:	learn: 0.4846358	total: 273ms	remaining: 17.9s
15:	learn: 0.4789874	total: 281ms	remaining: 17.3s
16:	learn: 0.4734691	total: 292ms	remaining: 16.9s
17:	learn: 0.4682772	total: 301ms	remaining: 16.4s
18:	learn: 0.46

Model:  CatBoost
Max KS:  30.766565213252996
\n


In [29]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import make_scorer

def validate_model_cv(X, y, model, scoring='ks'):

    cv = StratifiedKFold(
        n_splits=5,
        shuffle=True,
        random_state=25
    )

    if scoring == 'roc_auc':
        scorer = 'roc_auc'
    elif scoring == 'ks':
        scorer = ks_scorer
    else:
        raise ValueError("Unsupported scoring metric")


    scores = cross_val_score(
        model,
        X,
        y,
        cv=cv,
        scoring=scorer,
        n_jobs=-1
    )

    print(
        f"{model.__class__.__name__} | "
        f"{scoring}: {scores.mean():.4f} ± {scores.std():.4f}"
    )

    return scores.mean()


In [ ]:
pos_weight = (y_train == 0).sum() / (y_train == 1).sum()

xgb_unweighted = XGBClassifier(
    # eval_metric='logloss',
    # max_depth=3,
    # learning_rate=0.05,
    # n_estimators=300,
    random_state=25
)

xgb_weighted = XGBClassifier(
    # eval_metric='logloss',
    scale_pos_weight=pos_weight,
    # max_depth=3,
    # learning_rate=0.05,
    # n_estimators=300,
    random_state=25
)

lgb_unweighted = LGBMClassifier(random_state=25)

lgb_weighted = LGBMClassifier(
    class_weight='balanced',
    random_state=25
)


cb_unweighted = CatBoostClassifier(
    verbose=0,
    random_state=25
)

cb_weighted = CatBoostClassifier(
    auto_class_weights='Balanced',
    verbose=0,
    random_state=25
)


In [31]:
models_list = []
ks_scores = []

for name, model in {
    "XGB_Unweighted": xgb_unweighted,
    "XGB_Weighted": xgb_weighted,
    "LGB_Unweighted": lgb_unweighted,
    "LGB_Weighted": lgb_weighted,
    "Cat_Unweighted": cb_unweighted,
    "Cat_Weighted": cb_weighted
}.items():
    print(name)
    models_list.append(name)
    ks_scores.append(validate_model_cv(
        X_train,
        y_train,
        model,
        scoring='ks'
    ))

pd.DataFrame(data=ks_scores, index=models_list, columns=['mean'])

XGB_Unweighted
XGBClassifier | ks: 27.7574 ± 1.0813
XGB_Weighted
XGBClassifier | ks: 27.7574 ± 1.0813
LGB_Unweighted
LGBMClassifier | ks: 31.7882 ± 1.4741
LGB_Weighted
LGBMClassifier | ks: 31.5287 ± 1.6549
Cat_Unweighted
CatBoostClassifier | ks: 31.7651 ± 1.8242
Cat_Weighted
CatBoostClassifier | ks: 30.8774 ± 1.5702


,mean
XGB_Unweighted,27.757435
XGB_Weighted,27.757435
LGB_Unweighted,31.788184
LGB_Weighted,31.528666
Cat_Unweighted,31.765081
Cat_Weighted,30.877352


In [33]:
# we can see that unweighted models have more perfornamce than weighted, 
# we are going with champion and a challenger

In [32]:
cv = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=25
)

In [34]:

# Define parameter grids for grid search

# param_lgbm = {
#     'n_estimators': [100, 200, 500]
#     # 'learning_rate': [0.01, 0.05, 0.1],
#     # 'max_depth': [3, 5, 7],
#     # 'num_leaves': [15, 31, 63],
#     # 'feature_fraction': [0.6, 0.8, 1.0],
#     # 'bagging_fraction': [0.6, 0.8, 1.0],
# }

param_lgbm = {
    'max_depth': [3, 5],
    'num_leaves': [31, 63],
    'learning_rate': [0.05],
    'n_estimators': [200, 300],
    'feature_fraction': [0.8],
    'bagging_fraction': [0.8]
}


# GridSearchCV for each model

grid_lgbm = GridSearchCV(
    estimator=LGBMClassifier(
        boosting_type='gbdt',
        random_state=25
    ),
    param_grid=param_lgbm,
    scoring=ks_scorer,     # IMPORTANT
    cv=cv,
    n_jobs=-1,
    verbose=1
)

# Fitting the models

grid_lgbm.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Info] Number of positive: 4996, number of negative: 26309
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006247 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2924
[LightGBM] [Info] Number of data points in the train set: 31305, number of used features: 135
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.159591 -> initscore=-1.661273
[LightGBM] [Info] 

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=25, shuffle=True),
             estimator=LGBMClassifier(random_state=25), n_jobs=-1,
             param_grid={'bagging_fraction': [0.8], 'feature_fraction': [0.8],
                         'learning_rate': [0.05], 'max_depth': [3, 5],
                         'n_estimators': [200, 300], 'num_leaves': [31, 63]},
             scoring=<function ks_scorer at 0x0000014FF6F19B20>, verbose=1)

In [ ]:
grid_lgbm.best_params_
grid_lgbm.best_score_

{'bagging_fraction': 0.8,
 'feature_fraction': 0.8,
 'learning_rate': 0.05,
 'max_depth': 3,
 'n_estimators': 300,
 'num_leaves': 31}

np.float64(33.38664994606646)

In [38]:
param_cat = {
    'depth': [4, 6],
    'learning_rate': [0.05],
    'iterations': [300, 500],
    'l2_leaf_reg': [3, 5]
}


grid_cat = GridSearchCV(
    estimator=CatBoostClassifier(
        loss_function='Logloss',
        verbose=0,
        random_state=25
    ),
    param_grid=param_cat,
    scoring=ks_scorer,
    cv=cv,
    n_jobs=-1,
    verbose=1
)

grid_cat.fit(X_train, y_train)

grid_cat.best_params_
grid_cat.best_score_

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=25, shuffle=True),
             estimator=<catboost.core.CatBoostClassifier object at 0x0000014F9A1894C0>,
             n_jobs=-1,
             param_grid={'depth': [4, 6], 'iterations': [300, 500],
                         'l2_leaf_reg': [3, 5], 'learning_rate': [0.05]},
             scoring=<function ks_scorer at 0x0000014FF6F19B20>, verbose=1)

{'depth': 4, 'iterations': 500, 'l2_leaf_reg': 3, 'learning_rate': 0.05}

np.float64(33.28786652967712)

In [39]:
param_xgb = {
    'max_depth': [2, 3, 4],
    'min_child_weight': [50, 100],
    'subsample': [0.8],
    'colsample_bytree': [0.8]
}

grid_xgb = GridSearchCV(
    XGBClassifier(
        objective='binary:logistic',
        eval_metric='logloss',
        learning_rate=0.05,
        n_estimators=300,
        random_state=25
    ),
    param_grid=param_xgb,
    cv=cv,
    scoring=ks_scorer,
    n_jobs=-1,
    verbose=1
)

grid_xgb.fit(X_train, y_train)
grid_xgb.best_params_
grid_xgb.best_score_

Fitting 5 folds for each of 6 candidates, totalling 30 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=25, shuffle=True),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='logloss', feature_types=None,
                                     feature_weights=None, gamma=None,
                                     grow_...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=300,
                                     n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.8], 'max_depth': [2, 3, 4],
                         'min_child_weight': [50, 100], 'subsample': [0.8]},
             scoring=<function ks_scorer at 0x0000014FF6F19B20>, verbose=1)

{'colsample_bytree': 0.8,
 'max_depth': 2,
 'min_child_weight': 50,
 'subsample': 0.8}

np.float64(33.08832466745968)

In [54]:
from xgboost.callback import EarlyStopping
from lightgbm import early_stopping, log_evaluation

In [60]:
final_cat = CatBoostClassifier(
    loss_function='Logloss',
    random_state=25,
    depth=4,
    iterations=2000,        # large → early stopping decides
    l2_leaf_reg=3,
    learning_rate=0.05,
    verbose=0
)

final_cat.fit(
    X_dev, y_dev,
    eval_set=[(X_val, y_val)],
    early_stopping_rounds=50
)


In [55]:
final_lgb = LGBMClassifier(
    boosting_type='gbdt',
    learning_rate=0.05,
    max_depth=3,
    num_leaves=31,
    feature_fraction=0.8,
    bagging_fraction=0.8,
    n_estimators=2000,
    random_state=25
)

final_lgb.fit(
    X_dev, y_dev,
    eval_set=[(X_val, y_val)],
    eval_metric='binary_logloss',
    callbacks=[
        early_stopping(stopping_rounds=50),
        log_evaluation(period=0)  # suppress logs
    ]
)


[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Info] Number of positive: 3747, number of negative: 19731
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003047 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2917
[LightGBM] [Info] Number of data points in the train set: 23478, number of used features: 135
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fract

LGBMClassifier(bagging_fraction=0.8, feature_fraction=0.8, learning_rate=0.05,
               max_depth=3, n_estimators=2000, random_state=25)

In [59]:
FINAL_XGB_PARAMS = dict(
    objective='binary:logistic',
    eval_metric='logloss',
    learning_rate=0.05,
    n_estimators=300,
    random_state=25,
    colsample_bytree = 0.8,
    max_depth = 2,
    min_child_weight = 50,
    subsample = 0.8,
    early_stopping_rounds=50
)

final_xgb = XGBClassifier(**FINAL_XGB_PARAMS)

final_xgb.fit(
    X_dev, y_dev,
    eval_set=[(X_val, y_val)],
    verbose=False
)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=50,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=50, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=None,
              num_parallel_tree=None, ...)

In [61]:
import xgboost as xgb
import numpy as np

def get_pred_proba(model, X):
    """
    Returns predicted probability of class=1 (bad)
    Works for:
    - xgboost.train() models
    - sklearn-style models (LGBM, CatBoost)
    """
    if isinstance(model, xgb.Booster):
        dmat = xgb.DMatrix(X)
        return model.predict(dmat)
    else:
        return model.predict_proba(X)[:, 1]


In [62]:
def evaluate_ks(model, X_dev, y_dev, X_val, y_val):
    """
    Computes KS on dev and validation sets
    """
    y_dev_pred = get_pred_proba(model, X_dev)
    y_val_pred = get_pred_proba(model, X_val)

    ks_dev = max_ks(y_dev.values, y_dev_pred)
    ks_val = max_ks(y_val.values, y_val_pred)

    return ks_dev, ks_val


In [63]:
import pandas as pd

models = {
    "XGBoost": final_xgb,   # native xgb.Booster
    "LightGBM": final_lgb,  # LGBMClassifier
    "CatBoost": final_cat   # CatBoostClassifier
}

results = []

for name, model in models.items():
    ks_dev, ks_val = evaluate_ks(
        model,
        X_dev, y_dev,
        X_val, y_val
    )

    results.append({
        "Model": name,
        "Dev_KS": ks_dev,
        "Val_KS": ks_val,
        "KS_Gap": ks_dev - ks_val
    })

ks_comparison_df = pd.DataFrame(results).sort_values(
    "Val_KS", ascending=False
)

ks_comparison_df


[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8


,Model,Dev_KS,Val_KS,KS_Gap
0,XGBoost,34.830423,32.344526,2.485897
2,CatBoost,38.793413,31.206431,7.586982
1,LightGBM,39.434311,30.880052,8.554258
